In [99]:
import pandas as pd

In [100]:
StoreUserMapping=pd.read_excel('StoreUserMapping.xlsx')


In [101]:
StoreUserMapping.head()

ISPStoreUserMappingId      ISPStoreCode        UserName  Year  Month Day1  \
0                 491672       RETN_179074  ISPMARSGT-0218  2025     12   No   
1                 491671     RETN_179074_1  ISPMARSGT-0218  2025     12   No   
2                 491670       RETN_176444  ISPMARSGT-0218  2025     12   No   
3                 491669       RETN_176824  ISPMARSGT-0218  2025     12   No   
4                 491668  1039086910_C0538  ISPMARSGT-0218  2025     12   No   

  Day2 Day3 Day4 Day5  ... Day23 Day24 Day25 Day26 Day27 Day28 Day29 Day30  \
0   No   No   No   No  ...    No    No    No    No    No    WO    No    No   
1   No   No   No   No  ...    No    No    No    No    No    WO    No    No   
2   No   No   No   No  ...    No    No    No    No    No    WO    No    No   
3   No   No   No   No  ...    No    No    No    No   Yes    WO    No    No   
4   No   No   No   No  ...    No    No    No    No    No    WO    No    No   

  Day31 ClientName  
0    No    Mars GT  
1    No    Mars GT  
2    No    Mars GT  
3    No    Mars GT  
4    No    Mars GT  

[5 rows x 37 columns]

In [102]:
StoreUserMapping = StoreUserMapping[
    ~StoreUserMapping['UserName'].str.contains('TEST|SUP', na=False)]


In [103]:
day_cols = [col for col in StoreUserMapping.columns if col.startswith('Day')]

StoreUserMapping_unpivot = StoreUserMapping.melt(
    id_vars=[
        'ISPStoreUserMappingId',
        'ISPStoreCode',
        'UserName',
        'Year',
        'Month'
    ],
    value_vars=day_cols,
    var_name='Day',
    value_name='Status'
)


In [104]:
StoreUserMapping_unpivot

ISPStoreUserMappingId      ISPStoreCode        UserName  Year  Month  \
0                      491672       RETN_179074  ISPMARSGT-0218  2025     12   
1                      491671     RETN_179074_1  ISPMARSGT-0218  2025     12   
2                      491670       RETN_176444  ISPMARSGT-0218  2025     12   
3                      491669       RETN_176824  ISPMARSGT-0218  2025     12   
4                      491668  1039086910_C0538  ISPMARSGT-0218  2025     12   
...                       ...               ...             ...   ...    ...   
251188                 479364      RETN_1460360  ISPMARSGT-0041  2025     12   
251189                 479363    10399988_C1882  ISPMARSGT-0169  2025     12   
251190                 479362    10399988_C1552  ISPMARSGT-0169  2025     12   
251191                 479361   10400253_N01247  ISPMARSGT-0165  2025     12   
251192                 479360   10399988_N02518  ISPMARSGT-0164  2025     12   

          Day Status  
0        Day1     No  
1        Day1     No  
2        Day1     No  
3        Day1     No  
4        Day1     No  
...       ...    ...  
251188  Day31     No  
251189  Day31     No  
251190  Day31     No  
251191  Day31     No  
251192  Day31     No  

[251193 rows x 7 columns]

In [105]:
StoreUserMapping_unpivot['day'] = (
    StoreUserMapping_unpivot['Day'].str.replace('Day', '', regex=False).astype(int)
)

StoreUserMapping_unpivot['Date'] = pd.to_datetime(
    StoreUserMapping_unpivot[['Year', 'Month', 'day']].rename(
        columns={'Year': 'year', 'Month': 'month'}
    ),
    errors='coerce'
)


In [106]:
StoreUserMapping_unpivot['Date'] = pd.to_datetime(
    StoreUserMapping_unpivot['Year'].astype(str) + '-' +
    StoreUserMapping_unpivot['Month'].astype(str) + '-' +
    StoreUserMapping_unpivot['Day'].str.replace('Day', '', regex=False),
    format='%Y-%m-%d',
    errors='coerce'
)


In [107]:
StoreUserMapping_unpivot['Date_ddmmyyyy'] = (
    StoreUserMapping_unpivot['Date'].dt.strftime('%d-%m-%Y')
)


In [108]:
StoreUserMapping_unpivot=StoreUserMapping_unpivot.drop(['Year','Month','Date','Day','day'],axis=1)

In [109]:
StoreUserMapping_unpivot=StoreUserMapping_unpivot.rename(columns={'Date_ddmmyyyy':'Date'})

In [110]:
StoreUserMapping_unpivot.head(2)

ISPStoreUserMappingId   ISPStoreCode        UserName Status        Date
0                 491672    RETN_179074  ISPMARSGT-0218     No  01-12-2025
1                 491671  RETN_179074_1  ISPMARSGT-0218     No  01-12-2025

In [111]:
import pyodbc

server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;

    """

In [112]:


# ensure Date column is datetime (not string)
StoreUserMapping_unpivot['Date'] = pd.to_datetime(
    StoreUserMapping_unpivot['Date'],
    dayfirst=True,
    errors='coerce'
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

insert_sql = """
INSERT INTO PowerBI_LISV2.LISReporting.StoreUserMapping
(
    ISPStoreUserMappingId,
    ISPStoreCode,
    UserName,
    Status,
    [Date]
)
VALUES (?, ?, ?, ?, ?)
"""

data = list(
    StoreUserMapping_unpivot[
        ['ISPStoreUserMappingId', 'ISPStoreCode', 'UserName', 'Status', 'Date']
    ].itertuples(index=False, name=None)
)

cursor.fast_executemany = True
cursor.executemany(insert_sql, data)

conn.commit()
cursor.close()
conn.close()


In [113]:
StoreUserMapping_unpivot[StoreUserMapping_unpivot['UserName'].str.contains('SUP')]


Empty DataFrame
Columns: [ISPStoreUserMappingId, ISPStoreCode, UserName, Status, Date]
Index: []

In [114]:
StoreUserMapping_unpivot

ISPStoreUserMappingId      ISPStoreCode        UserName Status  \
0                      491672       RETN_179074  ISPMARSGT-0218     No   
1                      491671     RETN_179074_1  ISPMARSGT-0218     No   
2                      491670       RETN_176444  ISPMARSGT-0218     No   
3                      491669       RETN_176824  ISPMARSGT-0218     No   
4                      491668  1039086910_C0538  ISPMARSGT-0218     No   
...                       ...               ...             ...    ...   
251188                 479364      RETN_1460360  ISPMARSGT-0041     No   
251189                 479363    10399988_C1882  ISPMARSGT-0169     No   
251190                 479362    10399988_C1552  ISPMARSGT-0169     No   
251191                 479361   10400253_N01247  ISPMARSGT-0165     No   
251192                 479360   10399988_N02518  ISPMARSGT-0164     No   

             Date  
0      2025-12-01  
1      2025-12-01  
2      2025-12-01  
3      2025-12-01  
4      2025-12-01  
...           ...  
251188 2025-12-31  
251189 2025-12-31  
251190 2025-12-31  
251191 2025-12-31  
251192 2025-12-31  

[251193 rows x 5 columns]

In [ ]:
USE PowerBI_LISV2;

WITH cte AS (
    SELECT
        UserName,
        LISStoreCode,
        AttendanceType,

        TRY_CONVERT(date, LEFT(VisitDate, 10), 105) AS VisitDate,

        CASE 
            WHEN TRY_CONVERT(date, LEFT(VisitDate, 10), 105) IS NOT NULL THEN 1
            ELSE 0
        END AS Covered,

        CASE 
            WHEN AttendanceType = 'Attendance' THEN 1
            ELSE 0
        END AS Merchandised,

        CASE 
            WHEN AttendanceType = 'Attendance'
                THEN 'Merchandised'
            ELSE NoPermissionReason
        END AS Reason,

        TRY_CONVERT(time(0), [CheckIn-Time])  AS CheckInTime,
        TRY_CONVERT(time(0), [CheckOut-Time]) AS CheckOutTime,
        TRY_CONVERT(time(0), [TimeDuration])  AS TimeDuration

    FROM MarsManagement.ISPStoreAttendance_NormalExport
)

SELECT
    PJP.*,
    cte.Covered,
    cte.Merchandised,
    cte.Reason,
    cte.CheckInTime,
    cte.CheckOutTime,
    cte.TimeDuration
FROM PowerBI_LISV2.LISReporting.StoreUserMapping AS PJP
LEFT JOIN cte
    ON PJP.ISPStoreCode = cte.LISStoreCode
    AND PJP.UserName    = cte.UserName
WHERE 
    PJP.UserName NOT LIKE '%test%'
    AND PJP.UserName NOT LIKE '%SUP%'
    AND PJP.UserName IS NOT NULL;


In [140]:
import pandas as pd
import pyodbc

# SQL Server connection details
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

# Create connection
conn = pyodbc.connect(conn_str)

# SQL Query
query = """
WITH cte AS (
    SELECT
        UserName,
        LISStoreCode,
        AttendanceType,

        TRY_CONVERT(date, LEFT(VisitDate, 10), 105) AS VisitDate,

        CASE 
            WHEN TRY_CONVERT(date, LEFT(VisitDate, 10), 105) IS NOT NULL THEN 1
            ELSE 0
        END AS Covered,

        CASE 
            WHEN AttendanceType = 'Attendance' THEN 1
            ELSE 0
        END AS Merchandised,

        CASE 
            WHEN AttendanceType = 'Attendance'
                THEN 'Merchandised'
            ELSE NoPermissionReason
        END AS Reason,

        TRY_CONVERT(time(0), [CheckIn-Time])  AS CheckInTime,
        TRY_CONVERT(time(0), [CheckOut-Time]) AS CheckOutTime,
        TRY_CONVERT(time(0), [TimeDuration])  AS TimeDuration

    FROM MarsManagement.ISPStoreAttendance_NormalExport
   
)

SELECT
    PJP.*,
    cte.Covered,
    cte.Merchandised,
    cte.Reason,
    cte.CheckInTime,
    cte.CheckOutTime,
    cte.TimeDuration
FROM LISReporting.StoreUserMapping AS PJP
LEFT JOIN cte
    ON PJP.ISPStoreCode = cte.LISStoreCode
    AND PJP.UserName    = cte.UserName
WHERE 
    PJP.UserName NOT LIKE '%test%'
    AND PJP.UserName NOT LIKE '%SUP%'
    AND PJP.UserName IS NOT NULL;
"""

# Execute query and load into DataFrame
df_result = pd.read_sql(query, conn)

# Close connection
# conn.close()

# Preview result
df_result.head()


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8440\2567340379.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


ISPStoreUserMappingId    ISPStoreCode        UserName Status        Date  \
0                 481246  10329665_C0085  ISPMARSGT-0075     WO  2025-12-07   
1                 481246  10329665_C0085  ISPMARSGT-0075    Yes  2025-12-01   
2                 481246  10329665_C0085  ISPMARSGT-0075     No  2025-12-05   
3                 481246  10329665_C0085  ISPMARSGT-0075     No  2025-12-04   
4                 481246  10329665_C0085  ISPMARSGT-0075     No  2025-12-06   

   Covered  Merchandised                          Reason CheckInTime  \
0      1.0           0.0  Owner Temporarily Not Allowing    12:01:17   
1      1.0           0.0  Owner Temporarily Not Allowing    12:01:17   
2      1.0           0.0  Owner Temporarily Not Allowing    12:01:17   
3      1.0           0.0  Owner Temporarily Not Allowing    12:01:17   
4      1.0           0.0  Owner Temporarily Not Allowing    12:01:17   

  CheckOutTime TimeDuration  
0     12:01:17     00:00:00  
1     12:01:17     00:00:00  
2     12:01:17     00:00:00  
3     12:01:17     00:00:00  
4     12:01:17     00:00:00

In [122]:
df_result[df_result['UserName'].str.contains('SUP')]

Empty DataFrame
Columns: [ISPStoreUserMappingId, ISPStoreCode, UserName, Status, Date, Covered, Merchandised, Reason, CheckInTime, CheckOutTime, TimeDuration]
Index: []

In [154]:
query = """
SELECT *
FROM PowerBI_LISV2.LISReporting.StoreUserMapping
WHERE 
    Status = 'Yes'
    AND ISPStoreCode IS NOT NULL
    AND ISPStoreCode NOT LIKE '%test%'
"""

result = pd.read_sql(query, conn)

result.head()
result.to_excel('pjp.xlsx')


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8440\910657215.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [152]:
query = """
WITH RankedVisits AS (
    SELECT
        UserName,
        LISStoreCode,

        -- Safe Visit Date conversion
        TRY_CONVERT(date, LEFT(VisitDate, 10), 105) AS VisitDate,

        AttendanceType,
        PJPStatus,

        -- Time conversions
        TRY_CONVERT(time, [CheckIn-Time])  AS CheckInTime,
        TRY_CONVERT(time, [CheckOut-Time]) AS CheckOutTime,
        TRY_CONVERT(time, TimeDuration)    AS TimeDuration,

        -- Covered logic
        CASE 
            WHEN TRY_CONVERT(date, LEFT(VisitDate, 10), 105) IS NOT NULL THEN 1
            ELSE 0
        END AS Covered,

        -- Merchandised flag
        CASE 
            WHEN AttendanceType = 'Attendance' THEN 1
            ELSE 0
        END AS Merchandised,

        -- Final Status logic
        CASE 
            WHEN AttendanceType = 'Attendance'
                THEN 'Merchandised'
            ELSE NoPermissionReason
        END AS FinalStatus,

        -- Ranking logic
        ROW_NUMBER() OVER (
            PARTITION BY 
                UserName,
                LISStoreCode,
                TRY_CONVERT(date, LEFT(VisitDate, 10), 105)
            ORDER BY 
                TRY_CONVERT(time, [CheckIn-Time]) DESC
        ) AS rn

    FROM PowerBI_LISV2.MarsManagement.ISPStoreAttendance_NormalExport
    WHERE [month] = 12
      AND [year] = 2025
)

SELECT *
FROM RankedVisits
WHERE rn = 1
and UserName NOT LIKE '%test%'
    AND UserName NOT LIKE '%SUP%'
    AND UserName IS NOT NULL;

"""

result = pd.read_sql(query, conn)
result.head()


#result = pd.read_sql(query, conn)

#result.head()
result.to_excel('coverage.xlsx', index=False)

print("Excel file saved successfully as coverage.xlsx")


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_8440\3371465813.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


Excel file saved successfully as coverage.xlsx


In [131]:
result.columns


Index(['ISPStoreAttendance_NormalExportId', 'Month', 'Year', 'Region', 'State',
       'City', 'RSMCode', 'BranchCode', 'ASMCode', 'SOCode', 'SupervisorCode',
       'ISPCode', 'UserName', 'FullName', 'Designation', 'ClientStoreCode',
       'LISStoreCode', 'StoreName', 'StoreAddress', 'Format', 'ChainCode',
       'ChainName', 'SubChainName', 'StoreType', 'StoreClub',
       'DistributorCode', 'DistributorName', 'VisitDate', 'PJPStatus',
       'CheckIn-Time', 'CheckOut-Time', 'TimeDuration', 'AutoCheckOutStatus',
       'NoPermissionReason', 'InLatitude', 'InLongitude', 'OutLatitude',
       'OutLongitude', 'StoreCheck-InImageLink', 'StoreCheck-OutImageLink',
       'NoPermissionImageLink', 'AttendanceType', 'IsAudited'],
      dtype='object')